# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [19]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [20]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
print(df.shape)
print(df['Cabin'].nunique())
print(df['Ticket'].nunique())
# 類別型欄位 'Cabin' 和 數值型欄位 'Fare' 做群聚編碼
df['Cabin'] = df['Cabin'].fillna('None')
mean_df = df.groupby('Cabin')['Fare'].mean().reset_index()
median_df = df.groupby('Cabin')['Fare'].median().reset_index()
temp_df = pd.merge(mean_df, median_df, on='Cabin', how='left')
print(temp_df.head())
temp_df.columns = ['Cabin', 'Cabin_Fare_Mean', 'Cabin_Fare_Median']
df = pd.merge(df, temp_df, on='Cabin')
df


(891, 10)
147
681
  Cabin   Fare_x   Fare_y
0   A10  40.1250  40.1250
1   A14  52.0000  52.0000
2   A16  39.6000  39.6000
3   A19  26.0000  26.0000
4   A20  56.9292  56.9292


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_Fare_Mean,Cabin_Fare_Median
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S,19.157325,10.5000
1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,19.157325,10.5000
2,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S,19.157325,10.5000
3,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,None,Q,19.157325,10.5000
4,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,None,S,19.157325,10.5000
...,...,...,...,...,...,...,...,...,...,...,...,...
886,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,26.550000,26.5500
887,1,"Roebling, Mr. Washington Augustus II",male,31.0,0,0,PC 17590,50.4958,A24,S,50.495800,50.4958
888,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,83.158300,83.1583
889,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,30.000000,30.0000


In [21]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

7 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_Fare_Mean', 'Cabin_Fare_Median']



,Pclass,Age,SibSp,Parch,Fare,Cabin_Fare_Mean,Cabin_Fare_Median
0,3,22.0,1,0,7.2500,19.157325,10.5
1,3,26.0,0,0,7.9250,19.157325,10.5
2,3,35.0,0,0,8.0500,19.157325,10.5
3,3,-1.0,0,0,8.4583,19.157325,10.5
4,3,2.0,3,1,21.0750,19.157325,10.5


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?
> 分數有稍微提高  
代表生存與否和票價高低有一點相關

In [23]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
temp_df = df.drop(['Cabin_Fare_Mean', 'Cabin_Fare_Median'], axis=1)
estimator = LogisticRegression()
print("Logistic Regression Score: {}".format(cross_val_score(estimator, temp_df, train_Y, cv=5).mean()))

Logistic Regression Score: 0.6093979438872511


In [24]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
estimator = LogisticRegression()
print("Logistic Regression Score: {}".format(cross_val_score(estimator, df, train_Y, cv=5).mean()))

Logistic Regression Score: 0.6150349654069617
